In [0]:
profile="moth"

In [0]:
qhost = !qhost | grep godel

In [0]:
controller_script = "/tmp/ipcontroller.sh"
with open(controller_script, "w") as o:
    o.write(". activate conda\nipcontroller --profile=%s --debug\n" % profile)
!chmod +x $controller_script

In [0]:
!qdel ipcontroller
!qdel ipengine

In [0]:
exclude_ipengine = {"godel97", "godel199"}
import time, tempfile, os
cmd = "qsub -o /tmp -e /tmp -cwd -V -N ipcontroller -q godel199@godel199 %s" % controller_script
print cmd
!$cmd
time.sleep(10)
temp_files = []
for elem in [x.split() for x in qhost]:
    host = elem[0]
    if not host in exclude_ipengine:
        cpus = int(elem[2])
        load = elem[3]
        if cpus >= 8 and load != "-":
            tmp = tempfile.NamedTemporaryFile(delete=False)
            temp_files.append(tmp.name)
            tmp.write("bash -c 'pkill -9 ipengine'\n")
            for i in xrange(cpus):
                tmp.write(". activate conda\nipengine --profile=%s&\nsleep 5\n" % profile)
            tmp.write("sleep 365d\n")
            tmp.close()
            q = "all.q"
            if host == "godel97" or host == "godel199":
                q = "godel199"
            cmd = "qsub -o /tmp -e /tmp -cwd -V -N ipengine -q %s@%s -l mem_free=10g %s" % (q, host, tmp.name)
            print cmd
            !chmod +x {tmp.name}
            !$cmd

##Check size of the cluster

In [0]:
cpu_dict = {}
for elem in qhost:
    d = [x.strip() for x in elem.split()]
    cpu_dict[d[0]] = int(d[2])
qstat = !qstat | grep godel
total_cpus = 0
for elem in qstat:
    d = [x.strip() for x in elem.split()]
#     print d
    host = d[-2].split("@")[-1]
    if d[2] == 'ipengine':
        total_cpus += cpu_dict[host]
total_cpus

In [0]:
qstat  =!qstat | grep ipengine | grep qw
print len(qstat)
for elem in qstat:
    d = [x.strip() for x in elem.split()]
    print d
    !qdel {d[0]}

In [0]:
from datetime import datetime
from IPython.parallel import Client
from IPython.display import clear_output
rc = Client(profile=profile)
dview = rc[:]
print len(dview)

In [0]:
num = 0
while (1):
    print datetime.now().time(), len(rc[:])
    time.sleep(3)
    num += 1
    if num == 10:
        num = 0
        clear_output()

## Kill ipengines

In [0]:
clush_hosts = "'godel[02-37,26,33,199,97,200]'"
clush = "clush -b -w %s -u 10 -t 20" % clush_hosts
clush_nob = "clush -w %s -u 10 -t 20" % clush_hosts

In [0]:
!$clush_nob ps ax | grep cfriedline | grep -v grep | grep ipengine

In [0]:
!qdel ipcontroller
!qdel ipengine

In [0]:
!$clush pkill -9 ipcontroller
!$clush pkill -9 ipengine

In [0]:
!$clush pkill -9 ms

In [0]:
import netifaces

In [0]:
eth0 = netifaces.ifaddresses('eth0')
for i in eth0:
    if '192' in eth0[i][0]['addr']:
        print eth0[i][0]['addr']

In [0]:
!$clush_nob ps aux | grep bowtie

In [0]:
!$clush_nob pkill -9 bowtie2

In [0]:
!$clush_nob pkill -9 bcftools